In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

df = pd.read_csv('coupang_filtering.csv', encoding='utf8')

In [2]:
df

,Unnamed: 0,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리
0,0,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,7%,24210,"(100g당 1,519원)",4.5,11262,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎..."
1,1,1,293421715,926408953,5302203368,마이셰프 밀푀유나베 & 칼국수,"마이셰프 밀푀유나베 & 칼국수, 1129g, 1개",16900,0,16900,"(100g당 1,497원)",4.5,49201,0,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기,만두,생면,육수,표고버섯,팽이버섯,채소,배추,청경채,깻잎"
2,2,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15639,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수..."
3,3,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",20810,28%,14900,"(100g당 1,717원)",5.0,36322,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스"
4,4,1,2202010823,3744622966,71729775756,미소프레쉬 6컵 푸짐한 대구탕,"미소프레쉬 6컵 푸짐한 대구탕, 1225g, 1개",14200,0,14200,"(100g당 1,159원)",4.5,12051,0,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물,채소,연두부,다시팩,소스"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,325,7,6410059410,13743233827,80994055849,쉐푸드 유자소스 멕시칸 파히타 밀키트 (냉동),"쉐푸드 유자소스 멕시칸 파히타 밀키트 (냉동), 655g, 1개",17900,0,17900,"(100g당 2,733원)",4.5,10,1,"또띠아, 소스, 올리브오일, 흰다리새우살, 닭가슴살, 채소(양파, 황파프리카, 홍파...","또띠아,소스,올리브오일,흰다리새우살,닭가슴살,채소,양파,황파프리카,홍파프리카,청피망"
318,326,7,6635938377,15150332499,82371861026,쿡킷 동대문식 닭한마리 2인분 (냉동),"쿡킷 동대문식 닭한마리 2인분 (냉동), 1150g, 1개",17910,11%,15900,"(100g당 1,383원)",4.5,260,1,"양념 닭고기, 채소(감자, 대파), 다진 마늘, 떡과 칼국수 면, 치킨스톡으로 구성...","채소,감자,대파,다진마늘,떡,칼국수면,치킨스톡"
319,327,7,6842831802,16276794016,83469464083,바다자리 생새우 갈릭버터쉬림프 1~2인분,"바다자리 생새우 갈릭버터쉬림프 1~2인분, 300g, 1개",21600,0,21600,"(100g당 7,200원)",5.0,3,1,"새우, 야채 세트(양파, 마늘), 건고추, 하얀 설탕, 올리브 오일, 버터 후레시,...","새우,양파,마늘,건고추,올리브 오일,허브 솔트"
320,328,7,6407831462,13729238438,80980206078,마이셰프 알찬 소고기 스키야키,"마이셰프 알찬 소고기 스키야키, 1070g, 1개",16900,5%,15900,"(100g당 1,486원)",4.5,207,1,"소고기, 스키야키 육수, 해물 유부주머니와 두부 스틱, 폰즈 소스, 우동사리, 야채...","소고기,폰즈 소스,우동사리,야채,청경채,배추,대파,숙주나물,팽이버섯"


In [17]:
def category_name(product):
    if ('닭' in product)or('백숙' in product): name = '닭'
    elif '구이' in product : name = '구이'
    elif ('전골' in product)or('나베' in product)or('스키야키' in product)or('샤브샤브' in product): name = '전골'
    elif '해물탕' in product : name = '해물탕'
    elif '매운탕' in product : name = '매운탕'
    elif ('알탕' in product) : name = '알탕'

    elif ('면' in product)or('소바' in product)or('짜장' in product)or('누들' in product)or('짬뽕' in product) : name = '면'
    elif ('우동' in product)or('라면' in product) : name = '면'
    elif '국수' in product : name = '국수'

    elif ('찌개' in product) or ('짜글이' in product) : name = '찌개/짜글이'
    elif ('국' in product) or ('육개장' in product) or ('파개장' in product) : name = '국'

    elif ('파스타' in product) or ('라자냐' in product) : name = '파스타'
    elif '리조또' in product : name = '리조또'
        
    elif '탕' in product : name = '탕'
        
    elif ('떡볶이' in product) or ('떡볶잉' in product) : name = '떡볶이'
        
    elif '족발' in product : name = '족발'
    elif '순대' in product : name = '순대'
        
    elif '마라' in product : name = '마라'
    elif '마파두부' in product : name = '마파두부'
        
    elif '무침' in product : name = '무침'
    elif ('해물' in product)or('씨푸드' in product)or('씨키트' in product)or('꽃게' in product)or('새우' in product): name = '씨푸드'
    elif ('쉬림프' in product)or('연어' in product)or('조개' in product)or('문어' in product): name = '씨푸드'

    elif ('콩불' in product)or('콩나물 불고기' in product)or('잡채' in product)or('비빔밥' in product) : name='한식'
    elif ('부침개' in product)or('조림' in product)or('수제비' in product): name = '한식'

    elif '닭갈비' in product : name = '닭갈비'
    elif '오리' in product : name = '오리'
    elif '불고기' in product : name = '불고기'
    elif ('커리' in product) or ('카레' in product) : name = '카레'
    elif '찜' in product : name = '찜'
    elif '샌드위치' in product : name = '샌드위치'
    elif '쌈' in product : name = '쌈'
    elif '스테이크' in product : name = '스테이크'
    elif '감바스' in product : name = '감바스'
    elif '볶음' in product : name = '볶음'
    elif '덮밥' in product : name = '덮밥'

    else: name='기타'
        
    return name

In [18]:
# def category_name22(product):
#     if '면'|'누들'|'짜장'|'짬뽕' in product : 
#         name = '전골'
#     else : 
#         name = '기타'
#     return name

In [19]:
#df['상세카테고리']=df['상품'].apply(lambda x : category_name22(x))
df['상세카테고리']=df['상품'].apply(lambda x : category_name(x))
df['상세카테고리'].value_counts()

탕         36
전골        33
찌개/짜글이    33
파스타       28
국         19
면         19
씨푸드       18
스테이크      16
국수        12
한식        11
감바스       11
기타        10
닭         10
구이         9
알탕         9
찜          7
무침         7
매운탕        6
해물탕        6
떡볶이        6
쌈          3
마파두부       2
불고기        2
순대         2
샌드위치       2
리조또        2
카레         2
볶음         1
Name: 상세카테고리, dtype: int64

In [20]:
df.loc[df['카테고리명']==1].상세카테고리.value_counts()

찌개/짜글이    33
탕         32
전골        23
국         13
알탕         8
구이         6
매운탕        6
해물탕        4
면          4
닭          2
한식         1
찜          1
씨푸드        1
Name: 상세카테고리, dtype: int64

In [21]:
df.loc[df['카테고리명']==2].상세카테고리.value_counts()

씨푸드    2
Name: 상세카테고리, dtype: int64

In [22]:
df.loc[df['카테고리명']==3].상세카테고리.value_counts()

스테이크    9
쌈       1
Name: 상세카테고리, dtype: int64

In [23]:
df.loc[df['카테고리명']==4].상세카테고리.value_counts()

파스타    28
감바스     8
국수      8
면       6
씨푸드     3
한식      2
전골      1
해물탕     1
탕       1
기타      1
국       1
Name: 상세카테고리, dtype: int64

In [24]:
df.loc[df['카테고리명']==5].상세카테고리.value_counts()

떡볶이    2
순대     2
Name: 상세카테고리, dtype: int64

In [25]:
df.loc[df['카테고리명']==6].상세카테고리.value_counts()

한식      3
면       3
탕       3
마파두부    2
기타      1
Name: 상세카테고리, dtype: int64

In [26]:
df.loc[df['카테고리명']==7].상세카테고리.value_counts()

씨푸드     12
전골       9
기타       8
닭        8
무침       7
스테이크     7
찜        6
면        6
한식       5
국        5
떡볶이      4
국수       4
감바스      3
구이       3
쌈        2
리조또      2
카레       2
샌드위치     2
불고기      2
해물탕      1
알탕       1
볶음       1
Name: 상세카테고리, dtype: int64

In [32]:
print(df.loc[(df['카테고리명']==1)&(df['상세카테고리']=='탕'), '상품'])

4                                     미소프레쉬 6컵 푸짐한 대구탕
13     복선당 규스지 사골 도가니탕 2인분 1160g + 소스 20g x 2p 세트 (냉동)
19                                   앙트레 해물 누룽지탕 (2인분)
24                           바다자리 모둠조개탕 밀키트 2~3인분 550g
32                                    설래담 낙지 연포탕용 (냉동)
36              바다자리 겨울엔 생물 대구 맑은탕 씨키트 2~3인분 700g + 야채
39                                  식도락상점 꽃게탕 밀키트 (냉동)
41           마이셰프 TREASURE HUNTER 우정분식 물떡 어묵탕 2인분 (냉동)
44                                아라보감 산지직송 우럭지리탕용 씨키트
49                        바다자리 생물 바지락탕용 씨키트 2~3인분 500g
53             바다자리 겨울엔 생물 물메기 맑은탕 씨키트 2~3인분 700g + 야채
54                                         속초 동태탕 (냉동)
55                                   프레시지 통낙지 해신탕 (냉동)
57                                   프레시지 얼큰한 동태탕 (냉동)
59                                      속초 코다리맑음탕 (냉동)
62                                    오마뎅 그시절 오뎅탕 (냉동)
65                  바다자리 생물 아귀 맑은탕 씨키트 2~3인분 900g + 야채
66                           홈스토랑 부산식 물떡 & 꼬치 어묵탕 (냉동)
70        

In [34]:
df.loc[df['상품'].str.contains('맑은')]['상품']

36     바다자리 겨울엔 생물 대구 맑은탕 씨키트 2~3인분 700g + 야채
53    바다자리 겨울엔 생물 물메기 맑은탕 씨키트 2~3인분 700g + 야채
65         바다자리 생물 아귀 맑은탕 씨키트 2~3인분 900g + 야채
71                              빽쿡 맑은 국물 파육개장
Name: 상품, dtype: object

In [29]:
df.loc[df['카테고리명']==7].상품

222                                바다자리 보일링씨푸드 3~4인분
223               바다자리 국산깐새우 갈릭버터쉬림프 + 야채 씨키트 2인분 세트
224    바다자리 생굴 해물 굴부추전 씨키트 2~3인분 320g + 야채 + 부침가루 세트
225      바다자리 랍스터 보일링씨푸드 씨키트 3~4인분 950g + 야채 + 소스 세트
226            바다자리 매콤한 순살아귀 불고기 + 야채 + 소스 씨키트 2~3인분
                           ...                      
317                        쉐푸드 유자소스 멕시칸 파히타 밀키트 (냉동)
318                            쿡킷 동대문식 닭한마리 2인분 (냉동)
319                           바다자리 생새우 갈릭버터쉬림프 1~2인분
320                                 마이셰프 알찬 소고기 스키야키
321                            바다자리 생새우 보일링쉬림프 3~4인분
Name: 상품, Length: 100, dtype: object